In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, InputLayer, Input, add, dot, maximum, average, multiply, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
import numpy as np
import pandas as pd

In [2]:
# reading data
data = pd.read_csv('PrOCTOR_sample_data_all.csv', header=0)
data1 = data.fillna(data.mean()['MolecularWeight':'Salivary Gland'])
data1["target"] = np.where(data1.iloc[:, 1] == "passed", 1, 0)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# x, y variable
X = data1.iloc[:, 2:-1]
y = data1['target']
#print(X)

In [4]:
X = np.array(X)
X = scaler.fit_transform(X)
y = np.array(y)
y_1 = y.reshape(y.shape[0],-1)
#y = scaler.fit_transform(X)
des=X[:,:13]
body=X[:,13:]

In [5]:
print(des.shape)
print(body.shape)
print(y.shape)
print(y_1.shape)

(828, 13)
(828, 34)
(828,)
(828, 1)


In [6]:
hid_layer = [100,100,50]
output_layer = 1
output = 2
final_layer = 10

In [7]:
_, des_col = des.shape
_, body_col = body.shape

In [8]:
class MM_Concat:
    def __init__(self, des_shape, body_shape, hid_layer, final_layer ,output_layer , output):
        self.des_shape = des_shape
        self.body_shape = body_shape
        self.hid_layer = hid_layer  # not used
        self.final_layer = final_layer
        self.output_layer = output_layer
        self.output = output
        
    def Multi_modal_Sum(self):
        # des modality
        des_input = Input(shape = (self.des_shape,)) # input = 17, des_shape = 17
        des_input_x1_batch = BatchNormalization()(des_input)
        des_input_x2 = Dense(self.hid_layer[0], activation = 'relu')(des_input_x1_batch)
        des_input_x2_batch = BatchNormalization()(des_input_x2)
        des_input_x3 = Dense(self.hid_layer[1], activation = 'relu')(des_input_x2_batch)
        des_input_x3_batch = BatchNormalization()(des_input_x3)
        des_input_x4 = Dense(self.hid_layer[2], activation = 'relu')(des_input_x3_batch)
        des_input_x4_batch = BatchNormalization()(des_input_x4)
        des_output = Dense(self.output_layer , activation = 'sigmoid')(des_input_x4_batch)

        # body modality
        body_input = Input(shape = (self.body_shape,)) # input = 30,body_shape = 30
        body_input_x1_batch = BatchNormalization()(body_input)
        body_input_x2 = Dense(self.hid_layer[0], activation = 'relu')(body_input_x1_batch)
        body_input_x2_batch = BatchNormalization()(body_input_x2)
        body_input_x3 = Dense(self.hid_layer[1], activation = 'relu')(body_input_x2_batch)
        body_input_x3_batch = BatchNormalization()(body_input_x3)
        body_input_x4 = Dense(self.hid_layer[2], activation = 'relu')(body_input_x3_batch)
        body_input_x4_batch = BatchNormalization()(body_input_x4)
        body_output = Dense(self.output_layer , activation = 'sigmoid')(body_input_x4_batch)

        concat_layer = concatenate([des_output , body_output])
        f_layer = Dense(self.hid_layer[2], activation = 'relu')(concat_layer)
        concat_input_x = Dense(self.output_layer, activation = 'sigmoid')(f_layer)
        concat_model = keras.models.Model(inputs = [des_input, body_input], outputs = [concat_input_x])

        return concat_model

In [9]:
latefusion = MM_Concat(des_col , body_col , hid_layer , final_layer , output_layer , output)
lateFusion = latefusion.Multi_modal_Sum()
print(lateFusion)

In [10]:
lateFusion.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 13)           52          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 34)           136         input_2[0][0]                    
______________________________________________________________________________________________

In [92]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# 모델설정
sm = SMOTE(random_state=202004)
loo = LeaveOneOut()
kfold = KFold(n_splits = 10, shuffle = True, random_state = 111)
kfold.get_n_splits(X)

10

In [18]:
def predict_change(predict):
    y_predict = []
    for i in range(len(predict)):
        y_predict = np.append(y_predict, predict[i])
    
    y_predict = np.array(y_predict)
    y_predict = y_predict.reshape(y_predict.shape[0],-1)
    return y_predict

In [12]:
dict_y = dict()

class train_model:
    def __init__(self, x, y, model, pd_y, pd_sm_y, pd_cw_y, pd_smcw_y):
        self.X = x
        self.y = y
        self.model = model
        self.pd_y = pd_y
        self.pd_sm_y = pd_sm_y
        self.pd_cw_y = pd_cw_y
        self.pd_smcw_y = pd_smcw_y
        
    def train_based(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_y
    
    def train_smote(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_sm_y[index] = y_pred[i]
            
        return self.pd_sm_y
    
    def train_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_cw_y[index] = y_pred[i]
            
        return self.pd_cw_y
    
    def train_sm_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
    
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)
    
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_smcw_y[index] = y_pred[i]
            
        return self.pd_smcw_y

In [86]:
latefusion = MM_Concat(des_col , body_col , hid_layer , final_layer , output_layer , output)
lateFusion = latefusion.Multi_modal_Sum()
print(lateFusion)

late_y, late_sm_y, late_cw_y, late_smcw_y = [], [], [], []
output_late = train_model(X, y, lateFusion, dict_y, dict_y, dict_y, dict_y)

# concatenate model training
#late_y = output_late.train_based()
#late_sm_y = output_late.train_smote()
late_cw_y = output_late.train_weight()
#late_smcw_y = output_late.train_sm_weight()

TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
Epoch 1/50
21/21 [==============================] - 0s 2ms/step - loss: 0.6908 - accuracy: 0.1964
Epoch 2/50
21/21 [==============================] - 0s 1ms/step - loss: 0.6716 - accuracy: 0.4426
Epoch 3/50
21/21 [==============================] - 0s 1ms/step - loss: 0.6504 - accuracy: 0.5831
Epoch 4/50
21/21 [=======

21/21 [==============================] - 0s 1ms/step - loss: 0.1210 - accuracy: 0.9577
Epoch 44/50
21/21 [==============================] - 0s 1ms/step - loss: 0.0771 - accuracy: 0.9698
Epoch 45/50
21/21 [==============================] - 0s 1ms/step - loss: 0.0756 - accuracy: 0.9653
Epoch 46/50
21/21 [==============================] - 0s 997us/step - loss: 0.0913 - accuracy: 0.9547
Epoch 47/50
21/21 [==============================] - 0s 1ms/step - loss: 0.1081 - accuracy: 0.9622
Epoch 48/50
21/21 [==============================] - 0s 1ms/step - loss: 0.0875 - accuracy: 0.9517
Epoch 49/50
21/21 [==============================] - 0s 1ms/step - loss: 0.0878 - accuracy: 0.9653
Epoch 50/50
21/21 [==============================] - 0s 997us/step - loss: 0.1287 - accuracy: 0.9532
TEST: [  4  14  17  28  29  35  38  41  44  45  47  48  51  55  61  78  81  88
  90  92  95 108 109 114 116 124 127 128 142 148 151 155 159 161 163 168
 171 174 179 181 191 203 206 207 213 216 217 220 229 235 245 247

In [68]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math

class Evaluation:
    def __init__(self, pred, y):
        self.y_pred = pred
        self.y = y
        
    def matrix(self):
        y_1 = self.y.reshape(self.y.shape[0],-1)
        y_pred = np.array(self.y_pred)
        y_pred = y_pred.reshape(y_pred.shape[0], -1)
        
        y_classify = []
        for i in range(len(self.y_pred)):
            if self.y_pred[i] >= 0.5:
                a = 1.
                y_classify.append(a)
            else:
                a = 0.
                y_classify.append(a)
        
        fpr,tpr,threshold = roc_curve(y_1 , y_pred, pos_label = 1)
        precision, recall, threshold = precision_recall_curve(y_1, y_pred, pos_label = 1)
        
        roc_auc = auc(fpr,tpr)
        auprc = auc(recall, precision)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        F1 = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall)
        # binary 
        accuracy = accuracy_score(y_1, y_classify)
        mcc = matthews_corrcoef(y_1, y_classify) 
        g_mean = geometric_mean_score(y_1, y_classify)
        confusion = confusion_matrix(y_1, y_classify)
        print(confusion.ravel())
        tn, fp, fn, tp = confusion.ravel()
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = tp / (tp + fp)
        fnr = fn / (fn + tp)
        fpr = fp / (fp + tn)

        confu_precision = ppv 
        confu_recall = tpr # sensitivity
        confu_f1 = 2 * ((ppv * tpr) / (ppv + tpr))
        confu_accuracy = (tp + tn) / (tp + tn + fp + fn)
        confu_mcc = ((tp * tn)-(fp-fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        confu_g_mean = math.sqrt(tpr * tnr)
        Optimized_precision = (confu_accuracy - abs(tnr-tpr)) / (tnr + tpr)

        print('공통 \nAUC :',roc_auc) # pb
        print("AUPRC :", auprc) # pb
        print("Optimized precision :", Optimized_precision)

        print("\nfunction 사용\nAccuracy :", accuracy) #pb
        print("Precision(pb) :",mean_precision )
        print("Recall(pb) :", mean_recall) # pb
        print("F1 score(pb) :", F1) #pb

        print("MCC :", mcc)
        print("G-mean :", g_mean)


        print("\nConfusion_matrix 사용 \n", confusion)
        print("Accuracy :", confu_accuracy)
        print("Precision :", confu_precision) 
        print("Recall :", confu_recall) 
        print("F1 score :", confu_f1) 

        print("MCC :", confu_mcc)
        print("G-mean :", confu_g_mean)
        
        return roc_auc, auprc, Optimized_precision, accuracy, mean_precision, mean_recall, F1, mcc, g_mean, confu_accuracy, confu_precision, confu_recall, confu_f1, confu_mcc, confu_g_mean;

In [87]:
#late_y_result = Evaluation(predict_change(late_y), y)
#late_sm_y_result = Evaluation(late_sm_y, y)
late_cw_y_result = Evaluation(predict_change(late_cw_y), y)
#late_smcw_y_result = Evaluation(late_smcw_y, y)

print("#latefusion model 결과 \n")
print("\n-----based model-----\n")
#print(late_y_result.matrix())

print("\n-----smote model-----\n")
#print(late_sm_y_result.matrix())

print("\n-----class weight model-----\n")
print(late_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
#print(late_smcw_y_result.matrix())

#latefusion model 결과 


-----based model-----


-----smote model-----


-----class weight model-----

[ 43  28  65 692]
공통 
AUC : 0.854559324241353
AUPRC : 0.9822389466800856
Optimized precision : 0.38109764914807626

function 사용
Accuracy : 0.8876811594202898
Precision(pb) : 0.9796933714443589
Recall(pb) : 0.5310548152518244
F1 score(pb) : 0.6887592346062491
MCC : 0.432124591457609
G-mean : 0.7440637743697247

Confusion_matrix 사용 
 [[ 43  28]
 [ 65 692]]
Accuracy : 0.8876811594202898
Precision : 0.9611111111111111
Recall : 0.9141347424042272
F1 score : 0.937034529451591
MCC : 0.46084936831674345
G-mean : 0.7440637743697248
(0.854559324241353, 0.9822389466800856, 0.38109764914807626, 0.8876811594202898, 0.9796933714443589, 0.5310548152518244, 0.6887592346062491, 0.432124591457609, 0.7440637743697247, 0.8876811594202898, 0.9611111111111111, 0.9141347424042272, 0.937034529451591, 0.46084936831674345, 0.7440637743697248)

-----smote + class weight model-----



In [97]:
late_y_set = []

for i in range(20):
    latefusion = MM_Concat(des_col , body_col , hid_layer , final_layer , output_layer , output)
    lateFusion = latefusion.Multi_modal_Sum()
    print(lateFusion)

    late_y, late_sm_y, late_cw_y, late_smcw_y = [], [], [], []
    output_late = train_model(X, y, lateFusion, dict_y, dict_y, dict_y, dict_y)

    # concatenate model training
    #late_y = output_late.train_based()
    #late_sm_y = output_late.train_smote()
    #late_cw_y = output_late.train_weight()
    late_smcw_y = output_late.train_sm_weight()
    
    #late_y_result = Evaluation(predict_change(late_y), y)
    #late_sm_y_result = Evaluation(late_sm_y, y)
    #late_cw_y_result = Evaluation(predict_change(late_cw_y), y)
    late_smcw_y_result = Evaluation(predict_change(late_smcw_y), y)
    
    late_y_set = np.append(late_y_set, late_smcw_y_result.matrix())
    print(len(late_y_set))
    
late_y_np = np.array(late_y_set)

late_y_np = late_y_np.reshape(-1,15)
late_y_np.shape

TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
Epoch 1/50
43/43 [==============================] - 0s 2ms/step - loss: 2.1412 - accuracy: 0.5912
Epoch 2/50
43/43 [==============================] - 0s 1ms/step - loss: 1.6399 - accuracy: 0.5000
Epoch 3/50
43/43 [==============================] - 0s 1ms/step - loss: 1.1251 - accuracy: 0.5000
Epoch 4/50
43/43 [==============================] - 0s 1ms/step - loss: 0.8408 - accuracy: 0.5000
Epoch 5/50
43/43 [==============================] - 0s 1ms/step - loss: 0.7179 - accuracy: 0.5000
Epoch 6/50
43/43 [==============================] - 0s 1ms/step - loss: 0.6502 - accuracy: 0.5000
Epoch 7/50
43/43 [==============================] - 0s 1ms/step - lo

43/43 [==============================] - 0s 1ms/step - loss: 0.1016 - accuracy: 0.9657
Epoch 3/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0872 - accuracy: 0.9701
Epoch 4/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0955 - accuracy: 0.9687
Epoch 5/50
43/43 [==============================] - 0s 1ms/step - loss: 0.1215 - accuracy: 0.9599
Epoch 6/50
43/43 [==============================] - 0s 1ms/step - loss: 0.1429 - accuracy: 0.9497
Epoch 7/50
43/43 [==============================] - 0s 1ms/step - loss: 0.1131 - accuracy: 0.9592
Epoch 8/50
43/43 [==============================] - 0s 2ms/step - loss: 0.1166 - accuracy: 0.9621
Epoch 9/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0888 - accuracy: 0.9665
Epoch 10/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0998 - accuracy: 0.9592
Epoch 11/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0833 - accuracy: 0.9650
Epoch 12/50
43/43 [==========

43/43 [==============================] - 0s 1ms/step - loss: 0.0538 - accuracy: 0.9765
Epoch 8/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0741 - accuracy: 0.9735
Epoch 9/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0772 - accuracy: 0.9743
Epoch 10/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0481 - accuracy: 0.9801
Epoch 11/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0459 - accuracy: 0.9816
Epoch 12/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0371 - accuracy: 0.9853
Epoch 13/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0602 - accuracy: 0.9757
Epoch 14/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0703 - accuracy: 0.9757
Epoch 15/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0859 - accuracy: 0.9684
Epoch 16/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0853 - accuracy: 0.9647
Epoch 17/50
43/43 [=====

43/43 [==============================] - 0s 1ms/step - loss: 0.0453 - accuracy: 0.9853
Epoch 12/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0352 - accuracy: 0.9845
Epoch 13/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0367 - accuracy: 0.9838
Epoch 14/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0695 - accuracy: 0.9750
Epoch 15/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0553 - accuracy: 0.9816
Epoch 16/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0545 - accuracy: 0.9750
Epoch 17/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0371 - accuracy: 0.9831
Epoch 18/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0550 - accuracy: 0.9779
Epoch 19/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0429 - accuracy: 0.9882
Epoch 20/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0563 - accuracy: 0.9742
Epoch 21/50
43/43 [===

c:\users\user\anaconda3\envs\gpu\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

43/43 [==============================] - 0s 2ms/step - loss: 0.1247 - accuracy: 0.9614
Epoch 2/50
43/43 [==============================] - 0s 2ms/step - loss: 0.1226 - accuracy: 0.9628
Epoch 3/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0999 - accuracy: 0.9570
Epoch 4/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0896 - accuracy: 0.9679
Epoch 5/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0853 - accuracy: 0.9657
Epoch 6/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0990 - accuracy: 0.9708
Epoch 7/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0961 - accuracy: 0.9592
Epoch 8/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0888 - accuracy: 0.9679
Epoch 9/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0869 - accuracy: 0.9694
Epoch 10/50
43/43 [==============================] - 0s 2ms/step - loss: 0.1085 - accuracy: 0.9628
Epoch 11/50
43/43 [===========

43/43 [==============================] - 0s 3ms/step - loss: 0.1043 - accuracy: 0.9551
Epoch 7/50
43/43 [==============================] - 0s 3ms/step - loss: 0.0700 - accuracy: 0.9713
Epoch 8/50
43/43 [==============================] - 0s 3ms/step - loss: 0.0528 - accuracy: 0.9787
Epoch 9/50
43/43 [==============================] - 0s 3ms/step - loss: 0.1219 - accuracy: 0.9721
Epoch 10/50
43/43 [==============================] - 0s 3ms/step - loss: 0.1026 - accuracy: 0.9581
Epoch 11/50
43/43 [==============================] - 0s 3ms/step - loss: 0.1242 - accuracy: 0.9500
Epoch 12/50
43/43 [==============================] - 0s 3ms/step - loss: 0.0767 - accuracy: 0.9735
Epoch 13/50
43/43 [==============================] - 0s 3ms/step - loss: 0.0649 - accuracy: 0.9728
Epoch 14/50
43/43 [==============================] - 0s 3ms/step - loss: 0.0463 - accuracy: 0.9824
Epoch 15/50
43/43 [==============================] - 0s 3ms/step - loss: 0.0774 - accuracy: 0.9699
Epoch 16/50
43/43 [======

43/43 [==============================] - 0s 2ms/step - loss: 0.0468 - accuracy: 0.9816
Epoch 11/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0511 - accuracy: 0.9772
Epoch 12/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0710 - accuracy: 0.9698
Epoch 13/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0623 - accuracy: 0.9838
Epoch 14/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0668 - accuracy: 0.9713
Epoch 15/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0984 - accuracy: 0.9661
Epoch 16/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0755 - accuracy: 0.9661
Epoch 17/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0725 - accuracy: 0.9794
Epoch 18/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0628 - accuracy: 0.9779
Epoch 19/50
43/43 [==============================] - 0s 3ms/step - loss: 0.0718 - accuracy: 0.9735
Epoch 20/50
43/43 [===

c:\users\user\anaconda3\envs\gpu\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


(20, 15)

In [84]:
def result(data):
    auc = data[:,0]
    auprc = data[:,1]
    op = data[:,2]
    acc = data[:,3]
    precision = data[:,4]
    recall = data[:,5]
    f1 = data[:,6]
    mcc = data[:,7]
    g_mean = data[:,8]
    confu_acc = data[:,9]
    confu_precision = data[:,10]
    confu_recall = data[:,11]
    confu_f1 = data[:,12]
    confu_mcc = data[:,13]
    confu_g_mean = data[:,14]

    print('공통 \nAUC :\n',"[평균 :",format(auc.mean(), ".5f"), "] [표준편차 :",format(auc.std(), ".5f"),"]") # pb
    print("AUPRC :\n", "[평균 :",format(auprc.mean(), ".5f"), "] [표준편차 :",format(auprc.std(), ".5f"),"]")
    print("Optimized precision :\n","[평균 :",format(op.mean(), ".5f"), "] [표준편차 :",format(op.std(), ".5f"),"]")

    print("\nfunction 사용\nAccuracy :\n","[평균 :",format(acc.mean(), ".5f"), "] [표준편차 :",format(acc.std(), ".5f"),"]")
    print("Precision(pb) :\n", "[평균 :",format(precision.mean(), ".5f"), "] [표준편차 :",format(precision.std(), ".5f"),"]")
    print("Recall(pb) :\n", "[평균 :",format(recall.mean(), ".5f"), "] [표준편차 :",format(recall.std(), ".5f"),"]")
    print("F1 score(pb) :\n","[평균 :",format(f1.mean(), ".5f"), "] [표준편차 :",format(f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(mcc.mean(), ".5f"), "] [표준편차 :",format(mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(g_mean.mean(), ".5f"), "] [표준편차 :",format(g_mean.std(), ".5f"),"]")
    
    print("\nConfusion matrix 사용 \nAccuracy :\n", "[평균 :",format(confu_acc.mean(), ".5f"), "] [표준편차 :",format(confu_acc.std(), ".5f"),"]")
    print("Precision :\n", "[평균 :",format(confu_precision.mean(), ".5f"), "] [표준편차 :",format(confu_precision.std(), ".5f"),"]")
    print("Recall :\n", "[평균 :",format(confu_recall.mean(), ".5f"), "] [표준편차 :",format(confu_recall.std(), ".5f"),"]")
    print("F1 score :\n", "[평균 :",format(confu_f1.mean(), ".5f"), "] [표준편차 :",format(confu_f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(confu_mcc.mean(), ".5f"), "] [표준편차 :",format(confu_mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(confu_g_mean.mean(), ".5f"), "] [표준편차 :",format(confu_g_mean.std(), ".5f"),"]")
    

In [98]:
result(late_y_np)

공통 
AUC :
 [평균 : 0.93813 ] [표준편차 : 0.00572 ]
AUPRC :
 [평균 : 0.99312 ] [표준편차 : 0.00091 ]
Optimized precision :
 [평균 : 0.47444 ] [표준편차 : 0.01260 ]

function 사용
Accuracy :
 [평균 : 0.90048 ] [표준편차 : 0.00543 ]
Precision(pb) :
 [평균 : 0.99079 ] [표준편차 : 0.00100 ]
Recall(pb) :
 [평균 : 0.53323 ] [표준편차 : 0.00310 ]
F1 score(pb) :
 [평균 : 0.69332 ] [표준편차 : 0.00252 ]
MCC :
 [평균 : 0.57083 ] [표준편차 : 0.02031 ]
G-mean :
 [평균 : 0.86958 ] [표준편차 : 0.01771 ]

Confusion matrix 사용 
Accuracy :
 [평균 : 0.90048 ] [표준편차 : 0.00543 ]
Precision :
 [평균 : 0.98320 ] [표준편차 : 0.00363 ]
Recall :
 [평균 : 0.90667 ] [표준편차 : 0.00696 ]
F1 score :
 [평균 : 0.94336 ] [표준편차 : 0.00327 ]
MCC :
 [평균 : 0.58349 ] [표준편차 : 0.01808 ]
G-mean :
 [평균 : 0.86958 ] [표준편차 : 0.01771 ]
